In [22]:
import pandas as pd 
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import joblib

In [23]:
reviews_data = pd.read_excel("Hasil_Preprocessing.xlsx", usecols=["Label", "stemming"])
reviews_data.columns = ["label", "reviews"]

reviews_data.head()

,label,reviews
0,1,"['kemeja', 'bagus', 'banget', 'a', 'ngis', 'kn..."
1,0,"['jahitanya', 'rapi', 'cuman', 'benang', 'jahi..."
2,0,"['sesuai', 'harga', 'tipis', 'oke', 'warna', '..."
3,1,"['gila', 'bagus', 'worth', 'it', 'lembut', 'ba..."
4,0,"['kain', 'bagus', 'halus', 'buka', 'kotor', 'y..."


In [24]:
def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
reviews_data["reviews_join"] = reviews_data["reviews"].apply(join_text_list)

reviews_data["reviews_join"].head()

0    kemeja bagus banget a ngis knpa tidak dari dul...
1              jahitanya rapi cuman benang jahit jelek
2    sesuai harga tipis oke warna abu kalau di foto...
3    gila bagus worth it lembut baju kirain tipis s...
4           kain bagus halus buka kotor ya warna putih
Name: reviews_join, dtype: object

In [25]:
reviews_data

,label,reviews,reviews_join
0,1,"['kemeja', 'bagus', 'banget', 'a', 'ngis', 'kn...",kemeja bagus banget a ngis knpa tidak dari dul...
1,0,"['jahitanya', 'rapi', 'cuman', 'benang', 'jahi...",jahitanya rapi cuman benang jahit jelek
2,0,"['sesuai', 'harga', 'tipis', 'oke', 'warna', '...",sesuai harga tipis oke warna abu kalau di foto...
3,1,"['gila', 'bagus', 'worth', 'it', 'lembut', 'ba...",gila bagus worth it lembut baju kirain tipis s...
4,0,"['kain', 'bagus', 'halus', 'buka', 'kotor', 'y...",kain bagus halus buka kotor ya warna putih
...,...,...,...
826,1,"['terima', 'kasih', 'barang', 'sesuai', 'ukur'...",terima kasih barang sesuai ukur sesuai gambar ...
827,1,"['mantap', 'realpict', 'bangt', 'emas', 'cuman...",mantap realpict bangt emas cuman plastik aja s...
828,1,"['suka', 'banget', 'tas', 'tidak', 'seperti', ...",suka banget tas tidak seperti tas local keren ...
829,1,"['kualitas', 'produk', 'produk', 'original', '...",kualitas produk produk original harga produk


In [26]:
label = reviews_data["label"]
text = reviews_data["reviews_join"]

In [27]:
# Memisahkan data menjadi data train dan data test
train_data, test_data, train_labels, test_labels = train_test_split(text, label, test_size=0.2, random_state=42)

# Menggabungkan data train dan labels menjadi satu dataframe
train_df = pd.DataFrame({'text': train_data, 'label': train_labels})

# Menghitung jumlah data positif dan negatif pada data train
positive_count_train = (train_df['label'] == 1).sum()
negative_count_train = (train_df['label'] == 0).sum()

# Menentukan kelas dengan jumlah data terkecil
min_count = min(positive_count_train, negative_count_train)

# Mengambil sampel acak dari kelas positif dan negatif dengan ukuran yang sama
positive_samples = train_df[train_df['label'] == 1].sample(n=min_count, random_state=42)
negative_samples = train_df[train_df['label'] == 0].sample(n=min_count, random_state=42)

# Menggabungkan kembali sampel positif dan negatif menjadi satu dataframe
balanced_train_df = pd.concat([positive_samples, negative_samples])

# Memisahkan data dan labels pada data train yang seimbang
balanced_train_data = balanced_train_df['text']
balanced_train_labels = balanced_train_df['label']

# Menghitung rasio data positif dan negatif pada data train yang seimbang
total_count_train = len(balanced_train_labels)
positive_ratio_train = (balanced_train_labels == 1).sum() / total_count_train
negative_ratio_train = (balanced_train_labels == 0).sum() / total_count_train

# Menghitung jumlah data positif dan negatif pada data test
positive_count_test = (test_labels == 1).sum()
negative_count_test = (test_labels == 0).sum()

# Menghitung rasio data positif dan negatif pada data test
total_count_test = len(test_labels)
positive_ratio_test = positive_count_test / total_count_test
negative_ratio_test = negative_count_test / total_count_test

# Mencetak jumlah data training dan testing untuk kelas positif dan negatif
print("Jumlah data latih positif:", positive_count_train)
print("Jumlah data latih negatif:", negative_count_train)
print("Jumlah data uji positif:", positive_count_test)
print("Jumlah data uji negatif:", negative_count_test)


Jumlah data latih positif: 302
Jumlah data latih negatif: 362
Jumlah data uji positif: 83
Jumlah data uji negatif: 84


In [28]:
# Perhitungan TF vector pada train set
cvect = CountVectorizer()
TF_vector_train = cvect.fit_transform(train_data)

# Normalisasi TF vector pada train set
normalized_TF_vector_train = normalize(TF_vector_train, norm='l1', axis=1)

# Menampilkan output hasil TF
print("TF vector pada train set:")
print(TF_vector_train.toarray())

print("Normalized TF vector pada train set:")
print(normalized_TF_vector_train.toarray())


TF vector pada train set:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Normalized TF vector pada train set:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [29]:
# Perhitungan TF vector pada test set menggunakan CountVectorizer yang sudah dilatih pada train set
TF_vector_test = cvect.transform(test_data)

# Normalisasi TF vector pada test set
normalized_TF_vector_test = normalize(TF_vector_test, norm='l1', axis=1)

In [30]:
# Persentase fitur yang ingin dipilih setelah seleksi (10%)
percent = 30

# Menghitung jumlah fitur yang diinginkan berdasarkan persentase
k = int(percent / 100 * normalized_TF_vector_train.shape[1])

# Menerapkan seleksi fitur dengan chi-square pada train set
selector = SelectPercentile(chi2, percentile=percent)
tf_mat_train_selected = selector.fit_transform(normalized_TF_vector_train, train_labels)

# Mengaplikasikan seleksi fitur yang sama pada test set
tf_mat_test_selected = selector.transform(normalized_TF_vector_test)

# Menampilkan hasil seleksi fitur dengan chi-square pada train set
print("Hasil seleksi fitur dengan chi-square pada train set:")
print(tf_mat_train_selected.toarray())

# Menampilkan hasil seleksi fitur dengan chi-square pada test set
print("Hasil seleksi fitur dengan chi-square pada test set:")
print(tf_mat_test_selected.toarray())


Hasil seleksi fitur dengan chi-square pada train set:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Hasil seleksi fitur dengan chi-square pada test set:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [31]:
# Membuat objek model Multinomial Naive Bayes dengan class_prior yang sesuai
model = MultinomialNB(class_prior=[negative_ratio_train, positive_ratio_train])

# Melatih model dengan data latih yang sudah diseleksi
model.fit(tf_mat_train_selected, train_labels)

# Melakukan prediksi pada data uji yang sudah diseleksi
predictions = model.predict(tf_mat_test_selected)

prediksi_benar = (predictions == test_labels).sum()
prediksi_salah = (predictions != test_labels).sum()

# Menghitung jumlah data positif dan negatif pada data uji
positive_count_test = (test_labels == 1).sum()
negative_count_test = (test_labels == 0).sum()

# Mencetak jumlah data positif dan negatif pada data uji
print("Jumlah data uji positif :", positive_count_test)
print("Jumlah data uji negatif :", negative_count_test)

print('Jumlah prediksi benar\t:', prediksi_benar)
print('Jumlah prediksi salah\t:', prediksi_salah)

prediksi_positif = (predictions == 1).sum()
prediksi_negatif = (predictions == 0).sum()

print('Jumlah prediksi positif\t:', prediksi_positif)
print('Jumlah prediksi negatif\t:', prediksi_negatif)


# Menghitung akurasi
accuracy = accuracy_score(test_labels, predictions)
print("Akurasi:", accuracy)

# Menghitung precision
precision = precision_score(test_labels, predictions)
print("Precision:", precision)

# Menghitung recall
recall = recall_score(test_labels, predictions)
print("Recall:", recall)

# Menghitung F1-score
f1 = f1_score(test_labels, predictions)
print("F1-score:", f1)

Jumlah data uji positif : 83
Jumlah data uji negatif : 84
Jumlah prediksi benar	: 155
Jumlah prediksi salah	: 12
Jumlah prediksi positif	: 85
Jumlah prediksi negatif	: 82
Akurasi: 0.9281437125748503
Precision: 0.9176470588235294
Recall: 0.9397590361445783
F1-score: 0.9285714285714286


In [32]:
# Generate classification report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
# Menghitung confusion matrix
cm = confusion_matrix(test_labels, predictions)
print('Tabel Cofusion Matrix:\n', cm)
classification_rep = metrics.classification_report(test_labels, predictions, target_names=['negatif', 'positif'])
print('Classification Report:\n', classification_rep)

Tabel Cofusion Matrix:
 [[77  7]
 [ 5 78]]
Classification Report:
               precision    recall  f1-score   support

     negatif       0.94      0.92      0.93        84
     positif       0.92      0.94      0.93        83

    accuracy                           0.93       167
   macro avg       0.93      0.93      0.93       167
weighted avg       0.93      0.93      0.93       167



### Menyimpan hasil modeling

In [33]:

# Menyimpan model
joblib.dump(model, 'multinomial_nb_30 percent_model.pkl')

['multinomial_nb_30 percent_model.pkl']